In [1]:
import pandas as pd
from pprint import pprint
from pandas.io.json import json_normalize
import jsonlines
import numpy as np

In [2]:
df = pd.read_json("/workspace/tmp/diffs/ansible_ansible.jsonl", lines=True)

In [12]:
df.shape

(48112, 4)

In [13]:
def unnest(df, col, reset_index=False):
    col_flat = pd.DataFrame([[i, x] 
                       for i, y in df[col].apply(list).iteritems() 
                           for x in y], columns=['I', col])
    col_flat = col_flat.set_index('I')
    df = df.drop(col, 1)
    df = df.merge(col_flat, left_index=True, right_index=True)
    if reset_index:
        df = df.reset_index(drop=True)
    return df

In [53]:
df_test = df
df_test.shape

(48112, 4)

In [54]:
df_test[:5]

,sha,repository,message,modifications
0,f31421576b00f0b167cdbe61217c31c21a41ac02,ansible_ansible,Genesis.,"[{'added': 88, 'removed': 0, 'change_type': 'A..."
1,c4f6d493b3439f4f39c75b64fb9f8caca79485b7,ansible_ansible,Initial library directory,"[{'added': 5, 'removed': 0, 'change_type': 'AD..."
2,b9426ef99a35d9b0bddbb79c40b85c7d3dfd64b9,ansible_ansible,Add SSH-agent usage instructions,"[{'added': 2, 'removed': 0, 'change_type': 'MO..."
3,381b3c971ae3217b50462cdd82388802248ee1a0,ansible_ansible,Example for list of hosts syntax\n\nTODO: add ...,"[{'added': 2, 'removed': 0, 'change_type': 'AD..."
4,5170a9fe2c7d0d50ff2ae025fa475467696e525b,ansible_ansible,Readme edits,"[{'added': 31, 'removed': 18, 'change_type': '..."


In [67]:
df_test_flatten = unnest(df_test, "modifications")
df_test_flatten

,sha,repository,message,modifications
0,f31421576b00f0b167cdbe61217c31c21a41ac02,ansible_ansible,Genesis.,"{'added': 88, 'removed': 0, 'change_type': 'AD..."
0,f31421576b00f0b167cdbe61217c31c21a41ac02,ansible_ansible,Genesis.,"{'added': 165, 'removed': 0, 'change_type': 'A..."
1,c4f6d493b3439f4f39c75b64fb9f8caca79485b7,ansible_ansible,Initial library directory,"{'added': 5, 'removed': 0, 'change_type': 'ADD..."
2,b9426ef99a35d9b0bddbb79c40b85c7d3dfd64b9,ansible_ansible,Add SSH-agent usage instructions,"{'added': 2, 'removed': 0, 'change_type': 'MOD..."
3,381b3c971ae3217b50462cdd82388802248ee1a0,ansible_ansible,Example for list of hosts syntax\n\nTODO: add ...,"{'added': 2, 'removed': 0, 'change_type': 'ADD..."
...,...,...,...,...
48109,35a8922988841bb6efc190b9d020573d17736c8a,ansible_ansible,prepare_vmware_tests: datastore host is datast...,"{'added': 1, 'removed': 1, 'change_type': 'MOD..."
48110,257736e828c775c4ab4285b73a77f85de0fd550e,ansible_ansible,Update the number of contributors in README: 4...,"{'added': 1, 'removed': 1, 'change_type': 'MOD..."
48111,b475e0408c820e8b28cfc9aaf508e15761af0617,ansible_ansible,eos_lacp: Add unit testcases to eos_lacp (#644...,"{'added': 4, 'removed': 1, 'change_type': 'MOD..."
48111,b475e0408c820e8b28cfc9aaf508e15761af0617,ansible_ansible,eos_lacp: Add unit testcases to eos_lacp (#644...,"{'added': 1, 'removed': 0, 'change_type': 'ADD..."


In [68]:
df_test_normalized = json_normalize(df_test_flatten.modifications)
df_test_normalized

,added,removed,change_type,diff,filename,old_path,new_path
0,88,0,ADD,"@@ -0,0 +1,88 @@\n+Ansible\n+=======\n+\n+Ansi...",README.md,None,README.md
1,165,0,ADD,"@@ -0,0 +1,165 @@\n+# core\n+from optparse imp...",__init__.py,None,lib/ansible/__init__.py
2,5,0,ADD,"@@ -0,0 +1,5 @@\n+#!/usr/bin/python\n+\n+impor...",ping,None,library/ping
3,2,0,MODIFY,"@@ -35,6 +35,8 @@ Comamnd line usage example\n...",README.md,README.md,README.md
4,2,0,ADD,"@@ -0,0 +1,2 @@\n+127.0.0.1\n+192.168.1.255\n",ansible_hosts,None,examples/ansible_hosts
...,...,...,...,...,...,...,...
131717,1,1,MODIFY,"@@ -28,7 +28,7 @@\n validate_certs: Fa...",main.yml,test/integration/targets/vmware_content_librar...,test/integration/targets/vmware_content_librar...
131718,1,1,MODIFY,"@@ -76,7 +76,7 @@ Authors\n =======\n \n Ansib...",README.rst,README.rst,README.rst
131719,4,1,MODIFY,"@@ -37,6 +37,9 @@ class LacpFacts(object):\n \...",lacp.py,lib/ansible/module_utils/network/eos/facts/lac...,lib/ansible/module_utils/network/eos/facts/lac...
131720,1,0,ADD,"@@ -0,0 +1 @@\n+lacp system-priority 10\n",eos_lacp_config.cfg,None,test/units/modules/network/eos/fixtures/eos_la...


In [69]:
df_test_flatten = df_test_flatten.drop("modifications", 1)
df_test_joined = df_test_flatten.join(df_test_normalized)
df_test_joined

,sha,repository,message,added,removed,change_type,diff,filename,old_path,new_path
0,f31421576b00f0b167cdbe61217c31c21a41ac02,ansible_ansible,Genesis.,88,0,ADD,"@@ -0,0 +1,88 @@\n+Ansible\n+=======\n+\n+Ansi...",README.md,None,README.md
0,f31421576b00f0b167cdbe61217c31c21a41ac02,ansible_ansible,Genesis.,88,0,ADD,"@@ -0,0 +1,88 @@\n+Ansible\n+=======\n+\n+Ansi...",README.md,None,README.md
1,c4f6d493b3439f4f39c75b64fb9f8caca79485b7,ansible_ansible,Initial library directory,165,0,ADD,"@@ -0,0 +1,165 @@\n+# core\n+from optparse imp...",__init__.py,None,lib/ansible/__init__.py
2,b9426ef99a35d9b0bddbb79c40b85c7d3dfd64b9,ansible_ansible,Add SSH-agent usage instructions,5,0,ADD,"@@ -0,0 +1,5 @@\n+#!/usr/bin/python\n+\n+impor...",ping,None,library/ping
3,381b3c971ae3217b50462cdd82388802248ee1a0,ansible_ansible,Example for list of hosts syntax\n\nTODO: add ...,2,0,MODIFY,"@@ -35,6 +35,8 @@ Comamnd line usage example\n...",README.md,README.md,README.md
...,...,...,...,...,...,...,...,...,...,...
48109,35a8922988841bb6efc190b9d020573d17736c8a,ansible_ansible,prepare_vmware_tests: datastore host is datast...,0,2,MODIFY,"@@ -8,7 +8,6 @@ E125\n E126\n E129\n E131\n-E2...",legacy-ignore.txt,test/sanity/pep8/legacy-ignore.txt,test/sanity/pep8/legacy-ignore.txt
48110,257736e828c775c4ab4285b73a77f85de0fd550e,ansible_ansible,Update the number of contributors in README: 4...,1,1,MODIFY,"@@ -120,7 +120,7 @@ class TestGalaxy(unittest....",test_galaxy.py,test/units/cli/test_galaxy.py,test/units/cli/test_galaxy.py
48111,b475e0408c820e8b28cfc9aaf508e15761af0617,ansible_ansible,eos_lacp: Add unit testcases to eos_lacp (#644...,2,2,MODIFY,"@@ -135,7 +135,7 @@ class GCPAuthTestCase(unit...",test_auth.py,test/units/module_utils/gcp/test_auth.py,test/units/module_utils/gcp/test_auth.py
48111,b475e0408c820e8b28cfc9aaf508e15761af0617,ansible_ansible,eos_lacp: Add unit testcases to eos_lacp (#644...,2,2,MODIFY,"@@ -135,7 +135,7 @@ class GCPAuthTestCase(unit...",test_auth.py,test/units/module_utils/gcp/test_auth.py,test/units/module_utils/gcp/test_auth.py


In [58]:
# TODO: tokenize messages
df["message"].map(lambda x: len(x)).mean(), df["message"].map(lambda x: len(x)).max()

(138.91640339208513, 12220)

In [59]:
df.message.max()

'🚸 🐍 🍒 ⛏ Integrate cherry picker (#41403)\n\nThis enables developers to use cherry-picker for backporting purposes.\r\nThis tool originally comes from Core Python Development Workflow.\r\n\r\nRef: https://pypi.org/p/cherry-picker\r\nRef: https://github.com/python/core-workflow/tree/master/cherry_picker\r\n\r\nAlso:\r\n* 📝 Add docs about supporting cherry-picker'

In [60]:
f = df["modifications"][0][0]["diff"]
f

'@@ -0,0 +1,88 @@\n+Ansible\n+=======\n+\n+Ansible is a extra-simple Python API for doing \'remote things\' over SSH.  \n+\n+As Func, which I co-wrote, aspired to avoid using SSH and have it\'s own daemon infrastructure, Ansible aspires to be quite different and more minimal, but still able to grow more modularly over time. \n+\n+Principles\n+==========\n+\n+* Dead simple setup\n+* No server or client daemons, uses existing SSHd\n+* Only SSH keys are allowed for authentication\n+* usage of ssh-agent is more or less required\n+* plugins can be written in ANY language\n+* as with Func, API usage is an equal citizen to CLI usage\n+\n+Requirements\n+============\n+\n+* python 2.6 -- or a backport of the multiprocessing module\n+* paramiko\n+\n+Inventory file\n+==============\n+\n+The default inventory file (-H) is ~/.ansible_hosts and is a list\n+of all hostnames to target with ansible, one per line.  \n+\n+This list is further filtered by the pattern wildcard (-P) to target\n+specific hos

In [7]:
print(f)

@@ -0,0 +1,88 @@
+Ansible
+=======
+
+Ansible is a extra-simple Python API for doing 'remote things' over SSH.  
+
+As Func, which I co-wrote, aspired to avoid using SSH and have it's own daemon infrastructure, Ansible aspires to be quite different and more minimal, but still able to grow more modularly over time. 
+
+Principles
+==========
+
+* Dead simple setup
+* No server or client daemons, uses existing SSHd
+* Only SSH keys are allowed for authentication
+* usage of ssh-agent is more or less required
+* plugins can be written in ANY language
+* as with Func, API usage is an equal citizen to CLI usage
+
+Requirements
+============
+
+* python 2.6 -- or a backport of the multiprocessing module
+* paramiko
+
+Inventory file
+==============
+
+The default inventory file (-H) is ~/.ansible_hosts and is a list
+of all hostnames to target with ansible, one per line.  
+
+This list is further filtered by the pattern wildcard (-P) to target
+specific hosts.
+
+Comamnd line usage example
+

In [8]:
from abc import ABC
import typing
import nltk
import re
nltk.download('punkt')

class PatchParser(ABC):
    def parse(self, patch: str) -> typing.List[str]:
        pass
    
    
class NaivePatchParser(PatchParser):
    def __init__(self):
        self.punctuation = "!\"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"
        self.addition_start_token = "ADDS"
        self.deletion_start_token = "DELS"
        self.addition_end_token = "ADDE"
        self.deletion_end_token = "DELE"
        
        self.mwe_tokenizer = nltk.MWETokenizer(separator="")
        self.mwe_tokenizer.add_mwe(("<", self.addition_start_token, ">"))
        self.mwe_tokenizer.add_mwe(("<", self.deletion_start_token, ">"))
        self.mwe_tokenizer.add_mwe(("<", self.addition_end_token, ">"))
        self.mwe_tokenizer.add_mwe(("<", self.deletion_end_token, ">"))
        
    def parse(self, patch: str, ignore_punctuation: bool = False) -> typing.List[str]:
        prepared_string = patch.lower() \
            .replace("\n+", f"\n<{self.addition_start_token}>") \
            .replace("\n-", f"\n<{self.deletion_start_token}>")
        prepared_string = re.sub("\@@.*?\@@", "", prepared_string)
        prepared_string = re.sub(f"([{self.punctuation}])", r" \1 ", prepared_string)
        
        lines = prepared_string.split("\n")
        lines = [self.__prepare_line(line) for line in lines]
        tokens = nltk.word_tokenize("\n".join(lines))
        tokens = self.mwe_tokenizer.tokenize(tokens)
        
        if ignore_punctuation:
            tokens = [token for token in tokens if token not in self.punctuation]
        
        return tokens
    
    def __prepare_line(self, line: str) -> str:
        if line.startswith(f" < {self.addition_start_token} > "):
            return line + f" < {self.addition_end_token} > "
        if line.startswith(f" < {self.deletion_start_token} > "):
            return line + f" < {self.deletion_end_token} > "
        return line

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [9]:
f

'@@ -0,0 +1,88 @@\n+Ansible\n+=======\n+\n+Ansible is a extra-simple Python API for doing \'remote things\' over SSH.  \n+\n+As Func, which I co-wrote, aspired to avoid using SSH and have it\'s own daemon infrastructure, Ansible aspires to be quite different and more minimal, but still able to grow more modularly over time. \n+\n+Principles\n+==========\n+\n+* Dead simple setup\n+* No server or client daemons, uses existing SSHd\n+* Only SSH keys are allowed for authentication\n+* usage of ssh-agent is more or less required\n+* plugins can be written in ANY language\n+* as with Func, API usage is an equal citizen to CLI usage\n+\n+Requirements\n+============\n+\n+* python 2.6 -- or a backport of the multiprocessing module\n+* paramiko\n+\n+Inventory file\n+==============\n+\n+The default inventory file (-H) is ~/.ansible_hosts and is a list\n+of all hostnames to target with ansible, one per line.  \n+\n+This list is further filtered by the pattern wildcard (-P) to target\n+specific hos

In [10]:
parser = NaivePatchParser()
words = parser.parse(patch=f)
len(words)

830

In [11]:
words

['<ADDS>',
 'ansible',
 '<ADDE>',
 '<ADDS>',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '<ADDE>',
 '<ADDS>',
 '<ADDE>',
 '<ADDS>',
 'ansible',
 'is',
 'a',
 'extra',
 '-',
 'simple',
 'python',
 'api',
 'for',
 'doing',
 "'",
 'remote',
 'things',
 "'",
 'over',
 'ssh',
 '.',
 '<ADDE>',
 '<ADDS>',
 '<ADDE>',
 '<ADDS>',
 'as',
 'func',
 ',',
 'which',
 'i',
 'co',
 '-',
 'wrote',
 ',',
 'aspired',
 'to',
 'avoid',
 'using',
 'ssh',
 'and',
 'have',
 'it',
 "'",
 's',
 'own',
 'daemon',
 'infrastructure',
 ',',
 'ansible',
 'aspires',
 'to',
 'be',
 'quite',
 'different',
 'and',
 'more',
 'minimal',
 ',',
 'but',
 'still',
 'able',
 'to',
 'grow',
 'more',
 'modularly',
 'over',
 'time',
 '.',
 '<ADDE>',
 '<ADDS>',
 '<ADDE>',
 '<ADDS>',
 'principles',
 '<ADDE>',
 '<ADDS>',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '=',
 '<ADDE>',
 '<ADDS>',
 '<ADDE>',
 '<ADDS>',
 '*',
 'dead',
 'simple',
 'setup',
 '<ADDE>',
 '<ADDS>',
 '*',
 'no',
 'server',
 'or',
 'client',
 'daemons',
 

In [13]:
parser.parse(patch=f, ignore_punctuation=True)

['def',
 'detect',
 'type',
 'media',
 'self',
 'interfaces',
 'def',
 'get',
 'default',
 'interfaces',
 'self',
 'route',
 'path',
 'use',
 'the',
 'commands',
 '<DELS>',
 'route',
 'n',
 'get',
 '8',
 '8',
 '8',
 '8',
 'google',
 'public',
 'dns',
 '<DELE>',
 '<DELS>',
 'route',
 'n',
 'get',
 'inet6',
 '2404',
 '6800',
 '400a',
 '800',
 '1012',
 'ipv6',
 'google',
 'com',
 '<DELE>',
 '<ADDS>',
 'route',
 'n',
 'get',
 'default',
 '<ADDE>',
 '<ADDS>',
 'route',
 'n',
 'get',
 'inet6',
 'default',
 '<ADDE>',
 'to',
 'find',
 'out',
 'the',
 'default',
 'outgoing',
 'interface',
 'address',
 'and',
 'gateway',
 '<DELS>',
 'command',
 'dict',
 'v4',
 'route',
 'path',
 'n',
 'get',
 '8',
 '8',
 '8',
 '8',
 '<DELE>',
 '<DELS>',
 'v6',
 'route',
 'path',
 'n',
 'get',
 'inet6',
 '2404',
 '6800',
 '400a',
 '800',
 '1012',
 '<DELE>',
 '<ADDS>',
 'command',
 'dict',
 'v4',
 'route',
 'path',
 'n',
 'get',
 'default',
 '<ADDE>',
 '<ADDS>',
 'v6',
 'route',
 'path',
 'n',
 'get',
 'inet6',
 '

In [ ]:
# Explore punctuation dependency on length

In [70]:
files1 = df_test_joined.fillna(value="")

In [83]:
import swifter
import numpy as np
from multiprocessing.pool import Pool

In [89]:
def tokenize_chunk(df):
    df["patch_tokens"] = df["diff"].swifter.apply(lambda x: NaivePatchParser().parse(x))
    return df

In [90]:
def parallelize_dataframe(df, func, n_cores=8):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [91]:
parallelize_dataframe(files1, tokenize_chunk)

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


/opt/conda/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


/opt/conda/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


/opt/conda/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


/opt/conda/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


...

/opt/conda/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


/opt/conda/lib/python3.7/site-packages/tqdm/std.py:648: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


Process ForkPoolWorker-11:
Process ForkPoolWorker-15:
Process ForkPoolWorker-16:
Process ForkPoolWorker-14:
Process ForkPoolWorker-18:
Process ForkPoolWorker-12:
Traceback (most recent call last):
Process ForkPoolWorker-17:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Process ForkPoolWorker-13:
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/opt/conda/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/opt/conda/lib/python3.7/multiprocessi

KeyboardInterrupt: 

KeyboardInterrupt
  File "/opt/conda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 351, in get
    with self._rlock:
KeyboardInterrupt
  File "/opt/conda/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/opt/conda/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/opt/conda/lib/python3.7/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)
  File "/opt/conda/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


In [19]:
files1.patch_tokens.map(lambda x: len(x)).mean()

672.0427024420185

In [20]:
files1.patch_tokens.map(lambda x: len(x)).quantile(0.5)

181.0

In [21]:
files1.patch_tokens.map(lambda x: len(x)).quantile(0.95)

2997.5999999999985

In [22]:
files1["patch_tokens_no_punct"] = files1.apply(lambda row: parser.parse(row["diff"], ignore_punctuation=True), axis=1)

In [23]:
files1.patch_tokens_no_punct.map(lambda x: len(x)).mean()

393.4379411635758

In [24]:
files1.patch_tokens_no_punct.map(lambda x: len(x)).quantile(0.5)

103.0

In [25]:
files1.patch_tokens_no_punct.map(lambda x: len(x)).quantile(0.95)

1820.0

In [ ]:
# Explore the repo modification characteristics

In [26]:
files1.changes.mean(), files1.changes.max(), files1.changes.quantile(0.95)

(50.456398789951336, 8278.0, 252.59999999999854)

In [27]:
files1.additions.mean(), files1.additions.max(), files1.additions.quantile(0.95)

(40.08615020386689, 4139.0, 229.0)

In [28]:
files1.deletions.mean(), files1.deletions.max(), files1.deletions.quantile(0.95)

(10.37024858608444, 4908.0, 38.0)

In [29]:
files1.groupby("status").aggregate(np.sum)

,additions,changes,deletions
status,,,
added,639699.0,639699.0,0.0
modified,266704.0,451834.0,185130.0
removed,0.0,45831.0,45831.0
renamed,7922.0,13496.0,5574.0


In [34]:
files1.groupby("extension").aggregate(np.sum).sort_values("changes", ascending=False)

,additions,changes,deletions
extension,,,
.py,699000.0,826802.0,127802.0
.yml,114925.0,160176.0,45251.0
.yaml,29488.0,35413.0,5925.0
.txt,17415.0,29201.0,11786.0
.css,4490.0,24989.0,20499.0
...,...,...,...
.eot,0.0,0.0,0.0
.ttf,0.0,0.0,0.0
.woff,0.0,0.0,0.0


### Compare meaning by History (every file patch is treated as a single commit) vs by Files (files changes in a commit get averaged and then the avg value is used to compare against the history)

In [35]:
files1["patch_tokens_len"] = files1.apply(lambda row: len(row["patch_tokens"]), axis=1)

In [36]:
files1["patch_tokens_len"].mean(), files1["patch_tokens_len"].max(), files1["patch_tokens_len"].quantile(0.95)

(672.0427024420185, 36695, 2997.5999999999985)

In [37]:
group1 = files1.groupby(["commit_sha"])['patch_tokens_len'].mean()
group1

commit_sha
00067c8d54e40fa564a1a8923857905a3dd99d77      29.000000
003c26de04d9d602a1c1d646011ec3783be4b85d    2265.000000
005487cba2eb3bfb7115056363656a159737de81      41.000000
00604d3f2ca82b8ede4a941b2e271831f817f1ba     387.333333
00691f43c58a139e796e170cd6c77949c122bbb1     114.000000
                                               ...     
ffce2d3d2ad35485f20bebdadc03e9a3872cf704     807.500000
ffce456de9dde6868507efc9859e6f785585e5ac      59.000000
ffd117132f7d61a154fb95723f11262c59c63c6d      88.500000
ffe8f45ddd7bfc83402bd07ed737c536c117a5fb     253.000000
fff11bde0df33190fca022bd85d13135c8df97d6    2803.000000
Name: patch_tokens_len, Length: 4930, dtype: float64

In [38]:
mean_commits = pd.DataFrame(group1).patch_tokens_len
mean_commits.mean(), mean_commits.max(), mean_commits.quantile(0.95)

(767.4520070906422, 19974.0, 3060.7333333333336)